## Trying some new models and exploring through Ensemble and averaging preds directly to only check the differences ...Only an Experiment with Ensembles

# Got inspiration from medium article : https://towardsdatascience.com/ensemble-models-5a62d4f4cb0c

More to do and apply in other notebooks,


### Scores  Mean Sq log Error
model 1 Random forrests = 0.20992 
---
model 2 Extra trees = 0.14494
---
model 3 Adaboost = 0.20521
---
model 4 Xgboost = 0.13636
---
model avg = 0.15602
---
model 2-4 Xgboost + Extratrees = 0.13573

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install xgboost

In [ ]:
df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

## DATA PLAY


In [ ]:
df.info()

In [ ]:
df_test.info()

## EDA and Data management

In [ ]:
low_data = ['MSSubClass','MSZoning','LandContour','Street','Alley','Utilities','LotConfig','LandSlope','Condition1','Condition2','BldgType','RoofStyle','RoofMatl','ExterCond','BsmtCond','BsmtExposure','BsmtFinType2','BsmtFinSF2','Heating','CentralAir','Electrical','Functional','GarageQual','GarageCond','PavedDrive','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','FireplaceQu','PoolQC','Fence','MiscVal','MiscFeature','MasVnrType','SaleType','SaleCondition']
df = df.drop(columns = low_data)
df_test = df_test.drop(columns = low_data)

In [ ]:
df.info()

In [ ]:
cats = ['LotShape','Neighborhood','HouseStyle','Exterior1st','Exterior2nd','ExterQual' ,'Foundation','BsmtQual','BsmtFinType1','HeatingQC','KitchenQual','GarageType','GarageFinish']

In [ ]:
df[cats] = df[cats].astype('string')
df[cats] = df[cats].astype('string')

## 'Cats' moving all over the place--- Strings wont help ... need of numbers now!

In [ ]:
from sklearn import preprocessing


class CategoricalFeatures:
    def __init__(self, df, categorical_features, encoding_type, handle_na=False):
        """
        df: pandas dataframe
        categorical_features: list of column names, e.g. ["ord_1", "nom_0"......]
        encoding_type: label, binary, ohe
        handle_na: True/False
        """
        self.df = df
        self.cat_feats = categorical_features
        self.enc_type = encoding_type
        self.handle_na = handle_na
        self.label_encoders = dict()
        self.binary_encoders = dict()
        self.ohe = None

        if self.handle_na:
            for c in self.cat_feats:
                self.df.loc[:, c] = self.df.loc[:, c].astype(str).fillna("-9999999")
        self.output_df = self.df.copy(deep=True)
    
    def _label_encoding(self):
        for c in self.cat_feats:
            lbl = preprocessing.LabelEncoder()
            lbl.fit(self.df[c].values)
            self.output_df.loc[:, c] = lbl.transform(self.df[c].values)
            self.label_encoders[c] = lbl
        return self.output_df
    
    def _label_binarization(self):
        for c in self.cat_feats:
            lbl = preprocessing.LabelBinarizer()
            lbl.fit(self.df[c].values)
            val = lbl.transform(self.df[c].values)
            self.output_df = self.output_df.drop(c, axis=1)
            for j in range(val.shape[1]):
                new_col_name = c + f"__bin_{j}"
                self.output_df[new_col_name] = val[:, j]
            self.binary_encoders[c] = lbl
        return self.output_df

    def _one_hot(self):
        ohe = preprocessing.OneHotEncoder()
        ohe.fit(self.df[self.cat_feats].values)
        return ohe.transform(self.df[self.cat_feats].values)

    def fit_transform(self):
        if self.enc_type == "label":
            return self._label_encoding()
        elif self.enc_type == "binary":
            return self._label_binarization()
        elif self.enc_type == "ohe":
            return self._one_hot()
        else:
            raise Exception("Encoding type not understood")
    
    def transform(self, dataframe):
        if self.handle_na:
            for c in self.cat_feats:
                dataframe.loc[:, c] = dataframe.loc[:, c].astype(str).fillna("-9999999")

        if self.enc_type == "label":
            for c, lbl in self.label_encoders.items():
                dataframe.loc[:, c] = lbl.transform(dataframe[c].values)
            return dataframe

        elif self.enc_type == "binary":
            for c, lbl in self.binary_encoders.items():
                val = lbl.transform(dataframe[c].values)
                dataframe = dataframe.drop(c, axis=1)
                
                for j in range(val.shape[1]):
                    new_col_name = c + f"__bin_{j}"
                    dataframe[new_col_name] = val[:, j]
            return dataframe

        elif self.enc_type == "ohe":
            return self.ohe(dataframe[self.cat_feats].values)
        
        else:
            raise Exception("Encoding type not understood")



df_test["SalePrice"] = -1
full_data = pd.concat([df, df_test])

#cols = [c for c in SaleCondition   if c not in ["id", "target"]]

cat_feats = CategoricalFeatures(full_data, 
                                categorical_features=cats, 
                                encoding_type="label",
                                handle_na=True)
full_data_transformed = cat_feats.fit_transform()

X = full_data_transformed[:len(df)]
X_test = full_data_transformed[len(df):]

In [ ]:
df.columns

In [ ]:
X.info()

### SCaLe Laid Data and Fill laid  the DATASES

In [ ]:
##filling remaining NULL/Nan  values and convert to float16
X = X.fillna(X.mean())
X_test = X_test.fillna(X_test.mean())

In [ ]:
scaling_cols =['LotFrontage', 'LotArea', 'LotShape', 'Neighborhood',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'Exterior1st', 'Exterior2nd', 'MasVnrArea', 'ExterQual', 'Foundation',
       'BsmtQual', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF',
       'HeatingQC', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'MoSold', 'YrSold']

## Ur  Robusted Boiz

In [ ]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
X[scaling_cols] = rs.fit_transform(X[scaling_cols])
X_test[scaling_cols] = rs.transform(X_test[scaling_cols])
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X[scaling_cols] = sc.fit_transform(X[scaling_cols])
# X_test[scaling_cols] = sc.transform(X_test[scaling_cols])


In [ ]:
len(X.columns)

In [ ]:
X_test = X_test.drop(columns = ['SalePrice'])

## Grid Search and Model creation ....       How you can practice you new moves ... the META-Models secrets are not in books anymore!

### Scores  Mean Sq log Error
model 1 Random forrests = 0.20992 


---


model 2 Extra trees = 0.14494


---


model 3 Adaboost = 0.20521


---

model 4 Xgboost = 0.13636


---
model avg = 0.15602

---
model 2-4 Xgboost + Extratrees = 0.13573

import xgboost as xg

In [ ]:
##################  finding best so temp vars
temp_X = X.drop(columns = 'SalePrice')
temp_y = X['SalePrice']

## 1.   First Random Forests

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
reg1 = RandomForestRegressor()
param_grid = [
               {'bootstrap': [False],'n_estimators': [3,10,30,90,200, 300, 600],
                'max_features':['auto'],
                'max_depth': [20, 30,None],
                'min_samples_leaf': [1, 2],
                'min_samples_split': [2, 5]}
               ]

grid_search = GridSearchCV(reg1,param_grid,cv = 5,scoring = 'neg_mean_squared_log_error')

In [ ]:
grid_search.fit(temp_X,temp_y)

In [ ]:
grid_search.best_estimator_

In [ ]:
# RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
#                       max_depth=None, max_features='auto', max_leaf_nodes=None,
#                       max_samples=None, min_impurity_decrease=0.0,
#                       min_impurity_split=None, min_samples_leaf=2,
#                       min_samples_split=2, min_weight_fraction_leaf=0.0,
#                       n_estimators=30, n_jobs=None, oob_score=False,
#                       random_state=None, verbose=0, warm_start=False)

## 2.Extra Trees Regressor

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
reg2 = ExtraTreesRegressor()
param_grid = [
               {'bootstrap': [True,False],'n_estimators': [3,10,30,90,200, 300, 600],
                'max_features':['auto'],
                'max_depth': [10, 20,None],
                'min_samples_leaf': [1, 2, 4],
                'min_samples_split': [2, 5],
                'oob_score':[False]}
               ]
grid_search = GridSearchCV(reg2,param_grid,cv = 5,scoring = 'neg_mean_squared_log_error')

In [ ]:
grid_search.fit(temp_X,temp_y)

In [ ]:
grid_search.best_estimator_

In [ ]:
# ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
#                     max_depth=20, max_features='auto', max_leaf_nodes=None,
#                     max_samples=None, min_impurity_decrease=0.0,
#                     min_impurity_split=None, min_samples_leaf=1,
#                     min_samples_split=5, min_weight_fraction_leaf=0.0,
#                     n_estimators=600, n_jobs=None, oob_score=False,
#                     random_state=None, verbose=0, warm_start=False)


## 3. AdaBoost Regressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
reg3 = AdaBoostRegressor()
param_grid = [
               {'n_estimators': [3,10,30,90,200, 300, 600, 800, 1000, 1200, 1400],
                'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1.0]}
               ]

grid_search = GridSearchCV(reg3,param_grid,cv = 5,scoring = 'neg_mean_squared_log_error')

In [ ]:
grid_search.fit(temp_X,temp_y)

In [ ]:
grid_search.best_estimator_

In [ ]:
# AdaBoostRegressor(base_estimator=None, learning_rate=0.1, loss='linear',
#                   n_estimators=300, random_state=None)

## 4. XGBoost Regressor

In [ ]:
import xgboost as xg
reg4 = xg.XGBRegressor()
param_grid = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
grid_search = GridSearchCV(reg4,param_grid,cv = 5,scoring = 'neg_mean_squared_log_error')

In [ ]:
grid_search.fit(temp_X,temp_y)

In [ ]:
grid_search.best_estimator_

In [ ]:
# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=0.8, gamma=0.3,
#              importance_type='gain', learning_rate=0.1, max_delta_step=0,
#              max_depth=4, min_child_weight=4, missing=None, n_estimators=100,
#              n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
#              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#              silent=None, subsample=1.0, verbosity=1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xg
reg1 = RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

reg2 = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=20, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=5, min_weight_fraction_leaf=0.0,
                    n_estimators=600, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

reg3 = AdaBoostRegressor(base_estimator=None, learning_rate=0.1, loss='linear',
                  n_estimators=300, random_state=None)

reg4 = xg.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.3,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=4, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1.0, verbosity=1)

reg1.fit(temp_X,temp_y)
reg2.fit(temp_X,temp_y)
reg3.fit(temp_X,temp_y)
reg4.fit(temp_X,temp_y)

pred1 = reg1.predict(X_test)
pred2 = reg2.predict(X_test)
pred3 = reg3.predict(X_test)
pred4 = reg4.predict(X_test)

pred_avg = (pred1 + pred2 + pred3 + pred4)/4


In [ ]:
pred1

## asdfghjkjhgfqwertyuiopasdfghjkl;zxcvbnm,./

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/house-price/sample_submission.csv')

In [ ]:
i = 0
for ans in pred_avg:
    submit.SalePrice[i] = ans
    i = i + 1

In [ ]:
submit.to_csv('submission_avg.csv')

### Scores  Mean Sq log Error
model 1 Random forrests = 0.20992 


---


model 2 Extra trees = 0.14494


---


model 3 Adaboost = 0.20521


---

model 4 Xgboost = 0.13636


---
model avg = 0.15602

---
model 2-4 Xgboost + Extratrees = 0.13573
